## Generate and modify IDPP path 

In [1]:
from pymatgen_diffusion.neb.pathfinder import IDPPSolver
from pymatgen.core import Structure
import numpy as np
import os
from datetime import datetime

In [15]:
# run IDPP
source = "li-graphene-in-plane"
dirname = "C:\\Liugy-wd\\IDPP_test\\%s\\" % source
init_struct = Structure.from_file(os.path.join(dirname, "CONTCAR-ini"), False)
final_struct = Structure.from_file(os.path.join(dirname, "CONTCAR-fin"), False)
obj = IDPPSolver.from_endpoints(endpoints=[init_struct, final_struct], nimages=7, sort_tol=0, pi_bond=0.0)
idpp_path = obj.run(maxiter=5000, tol=1e-5, gtol=1e-3, species=None)
result = idpp_path

element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:Li radius:1.34


In [13]:
# remove clash
import time
start_time = time.time()

method = 'decay'
clash_removed_path, attr_force_log, attr_index_log, rpl_force_log, rpl_index_log, disp_log = obj.clash_removal(obj.structures,step_size=0.05, max_iter=200,k_bonded=0.05,base_step=0.01, max_step=0.05, step_update_method=method)
# available_method = ('decay', 'expo', 'triangular')
result = clash_removed_path

print("--- %.02d seconds ---" % (time.time() - start_time))

10%
20%
30%
40%
50%
60%
70%
80%
90%
--- 249 seconds ---


In [17]:
# rerun
obj_rerun = IDPPSolver(clash_removed_path)
rerun_path = obj_rerun.rerun(maxiter = 5000, tol=1e-5, gtol=1e-3)
result = rerun_path

element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:Li radius:0.83


## Saving data

In [16]:
# create saving folder
now = datetime.now()
today = now.strftime("%Y%m%d")
path = os.path.join(dirname, today)
if not os.path.exists(path):
    os.mkdir(path)
i = 1
saveDir = os.path.join(path,"%s_test_%02d"%(today,i))
while(os.path.exists(saveDir)): 
    i += 1
    saveDir = os.path.join(path,"%s_test_%02d"%(today,i))
os.mkdir(saveDir)


In [17]:
# save result structures as POSCAR
for n,struct in enumerate(idpp_path):
    # os.mkdir("%s\\%02d" % (saveDir,n))
    struct.to(fmt="poscar",filename="%s\\POSCAR_%02d" % (saveDir,n))

In [ ]:
# save f_log and disp_log
np.savetxt(os.path.join(saveDir, "attr_force_log.txt"), attr_force_log)
np.savetxt(os.path.join(saveDir, "rpl_force_log.txt"), rpl_force_log)
np.savetxt(os.path.join(saveDir, "attr_index_log.txt"), attr_index_log, fmt='%d')
np.savetxt(os.path.join(saveDir, "rpl_index_log.txt"), rpl_index_log, fmt='%d')
np.savetxt(os.path.join(saveDir, "disp_log.txt"), disp_log)

In [ ]:
# output log
with open(os.path.join(saveDir, "log.txt"),"a" ) as f:
    log = "r = %d \nmax_bond_length = %f\nradius = %f\nmax_iteration = %d\ntolerance = %d\ngtolerance%d" % (5, 4.09, 1.44, 5000, 1e-5, 1e-3)
    f.write(log)

# save probes
with open(os.path.join(saveDir, "d_3-4.txt"),"w") as f:
    for i in probe1:
        f.write("%f\n" % i)
with open(os.path.join(saveDir, "d_3-9.txt"),"w") as f:
    for i in probe2:
        f.write("%f\n" % i)
with open(os.path.join(saveDir, "d_3-10.txt"),"w") as f:
    for i in probe3:
        f.write("%f\n" % i)

np.savetxt(os.path.join(saveDir, "d_26.txt"),d_26)

with open(os.path.join(saveDir, "neighbor_26.txt"),"w") as f:
    for i in range(len(n_26)):
        f.write("iteration %d \n" % i)
        for ni in n_26[i]:
            f.write(str(ni)+"\n")

## Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

path = saveDir
# path = "C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191029\\20191029_test_04"
attr_force = np.loadtxt("%s\\attr_force_log.txt" % path)
rpl_force = np.loadtxt("%s\\rpl_force_log.txt" % path)
disp = np.loadtxt("%s\\disp_log.txt" % path)
num_of_images = len(attr_force[0])
y_lb = 0
y_ub = 0.5

plt.figure(figsize=[6.4, 9])

ax1 = plt.subplot(311)
for i in range(num_of_images):
    plt.plot(attr_force[:,i],label='image %02d' % i)
ax1.legend(loc="upper right",fontsize='small')
ax1.set_title("attractive force")

plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = plt.subplot(312)
for i in range(num_of_images):
    plt.plot(rpl_force[:,i],label='image %02d' % i)
ax2.legend(loc="upper right",fontsize='small')
ax2.set_ylim(0,0.5)
ax2.set_title("repulsive force")
plt.setp(ax2.get_xticklabels(),visible=False)

ax3 = plt.subplot(313)
for i in range(num_of_images):
    plt.plot(disp[:,i],label='image %02d' % i)
ax3.legend(loc="upper right",fontsize='small')
ax3.set_title("displacement")

# plt.title("different step size dynamics (all initial step sizes are 0.005 ")
plt.tight_layout()
plt.savefig("%s\\%s.png"%(saveDir,method),dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

path = "C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191028"
log = "attr_force_log.txt"
attr_force_03 = np.loadtxt("%s\\20191028_test_01\\%s" % (path, log))
attr_force_04 = np.loadtxt("%s\\20191028_test_04\\%s" % (path, log))
attr_force_05 = np.loadtxt("%s\\20191028_test_05\\%s" % (path, log))
attr_force_06 = np.loadtxt("%s\\20191028_test_06\\%s" % (path, log))
attr_force_01 = np.loadtxt("C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191029\\20191029_test_02\\%s" % log)

x = np.arange(200)
y_lb = 0
y_ub = 0.5
plt.figure(figsize=[6.4, 15])

ax1 = plt.subplot(511)
plt.plot(x, attr_force_03)
ax1.set_title("decay 0.01")
ax1.set_ylim(y_lb, y_ub)
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = plt.subplot(512)
plt.plot(attr_force_04)
ax2.set_title("linear increase")
ax2.set_ylim(y_lb, y_ub)
plt.setp(ax2.get_xticklabels(),visible=False)

ax3 = plt.subplot(513)
plt.plot(attr_force_05)
ax3.set_title("exponentially increase *2")
ax3.set_ylim(y_lb, y_ub)
plt.setp(ax3.get_xticklabels(),visible=False)

ax4 = plt.subplot(514)
plt.plot(attr_force_06)
ax4.set_title("*1.2 or reject")
ax4.set_ylim(y_lb, y_ub)

ax5 = plt.subplot(515)
plt.plot(attr_force_01)
ax5.set_title("cyclic step size")
ax5.set_ylim(y_lb, y_ub)

# plt.title("different step size dynamics (all initial step sizes are 0.005 ")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def _get_triangular_step(iteration, half_period=5, base_step=0.05, max_step=0.1):
    """
    Given the inputs, calculates the step_size that should be applicable
    for this iteration using CLR technique from Anand Saha
    (http://teleported.in/posts/cyclic-learning-rate/).
    """
    cycle = np.floor(1 + iteration/(2 * half_period))
    x = np.abs(iteration/half_period - 2 * cycle + 1)
    step = base_step + (max_step - base_step) * np.maximum(0, (1-x))
    return step

x = np.arange(200)
y = np.array(list(map(_get_triangular_step, x)))
plt.plot(x,y)
plt.show()

## Testing

In [ ]:
# check d_26
count = 0
for i in d_26:
    for j in i:
        diff = abs(j - 2.88)
        if (diff < 0.01 ):
            count += 1
print(count)